In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

dataset_base_path = '/content/drive/MyDrive/WeatherDataset-Q3/WeatherDataset'

# List of weather conditions
weather_conditions = ['rain', 'hail', 'glaze', 'frost', 'fogsmog', 'dew', 'lightning', 'rainbow', 'rime', 'sandstorm', 'snow']


In [22]:
IMG_SIZE = (150, 150)  # Size used for resizing images
BATCH_SIZE = 32

# Data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,  # Set validation split
    horizontal_flip=True,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.5, 1.5]
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Create train and validation generators
train_generator = train_datagen.flow_from_directory(
    dataset_base_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=weather_conditions
)

validation_generator = train_datagen.flow_from_directory(
    dataset_base_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=weather_conditions
)

# Create test generator
test_generator = test_datagen.flow_from_directory(
    dataset_base_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=weather_conditions
)


Found 1153 images belonging to 11 classes.
Found 287 images belonging to 11 classes.
Found 1440 images belonging to 11 classes.


In [23]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Freeze the layers of the base model
base_model.trainable = False

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(len(weather_conditions), activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=x)


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [24]:
model.compile(optimizer=Adam(learning_rate=1e-4),  # Adjust learning rate as needed
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [25]:
history = model.fit(
    train_generator,
    epochs=10,  # Adjust epochs as needed
    validation_data=validation_generator
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


37/37 ━━━━━━━━━━━━━━━━━━━━ 111s 3s/step - accuracy: 0.5098 - loss: 1.4583 - val_accuracy: 0.8328 - val_loss: 0.5410
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step - accuracy: 0.8736 - loss: 0.4686 - val_accuracy: 0.8641 - val_loss: 0.4234
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.8833 - loss: 0.3950 - val_accuracy: 0.8885 - val_loss: 0.3597
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.9049 - loss: 0.3199 - val_accuracy: 0.8990 - val_loss: 0.3523
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 0.9123 - loss: 0.2798 - val_accuracy: 0.8990 - val_loss: 0.3248
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - accuracy: 0.9232 - loss: 0.2625 - val_accuracy: 0.8955 - val_loss: 0.3333
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.9296 - loss: 0.2268 - val_accuracy: 0.9233 - val_loss: 0.2852
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.9257 - loss: 0.2190 - val_accuracy: 0.8955 - val_loss: 0.307

In [26]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.2f}")



45/45 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.9180 - loss: 0.2553
Test accuracy: 0.92
